In [1]:
import os
import pickle
import copy
import numpy as np
from tqdm import tqdm

# 원본 파일
with open('../../data/motion3d/tr_aihub_sport_ts_30.pkl', 'rb') as f:
	datareader = pickle.load(f)

In [2]:
datareader.keys()

dict_keys(['train', 'test'])

In [3]:
datareader['train'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'joints_2.5d_image', '2.5d_factor', 'camera_name', 'action', 'source', 'frame', 'world_3d', 'cam_3d', 'cam_param'])

In [4]:
datareader['train']['joint_2d'].shape # (1559752, 17, 2) array
datareader['train']['confidence'].shape # (1559752, 17, 1) array
datareader['train']['joint3d_image'].shape # (1559752, 17, 3)
datareader['train']['camera_name'].shape # (1559752,) array
len(datareader['train']['source']) # 1559752 list

561194

In [5]:
datareader['test'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'joints_2.5d_image', '2.5d_factor', 'camera_name', 'action', 'source', 'frame', 'world_3d', 'cam_3d', 'cam_param'])

In [6]:
datareader['test']['joint_2d'].shape # (566920, 17, 2) array
datareader['test']['confidence'].shape # (566920, 17, 1) array
datareader['test']['joint3d_image'].shape # (566920, 17, 3) array
datareader['test']['joints_2.5d_image'].shape # (566920, 17, 3) array = 2.5d_factor * joint3d_image
datareader['test']['2.5d_factor'].shape # (566920,) array
datareader['test']['camera_name'].shape # (566920,) array
len(datareader['test']['action']) # 566920 list
len(datareader['test']['source']) # 566920 list

3407

In [8]:
(datareader['test']['joint3d_image'][0][0]*datareader['test']['2.5d_factor'][0] == datareader['test']['joints_2.5d_image'][0][0]).min()

True

In [9]:
datareader['test']['source'][2431], datareader['test']['frame'][2431]

('res_30_F160A_7', 667)

In [9]:
import os
import getpass

user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)
aihub_root = '/home/{}/Datasets/HAAI/AIHUB'.format(user)
ap_aihub_result_root = alphapose_root + "/examples/aihub_result"
alphapose_root, motionbert_root, aihub_root, ap_aihub_result_root

('/home/hrai/codes/AlphaPose',
 '/home/hrai/codes/MotionBERT',
 '/home/hrai/Datasets/HAAI/AIHUB',
 '/home/hrai/codes/AlphaPose/examples/aihub_result')

In [10]:
os.chdir(motionbert_root)
from custom_codes.test_utils import *

In [11]:
h36m_keypoints ={
    0 : 'Pelvis',
    1 : 'R_Hip',
    2 : 'R_Knee',
    3 : 'R_Ankle',
    4 : 'L_Hip',
    5 : 'L_Knee',
    6 : 'L_Ankle',
    7 : 'Torso',
    8 : 'Neck',
    9 : 'Nose',
    10 : 'Head',
    11 : 'L_Shoulder',
    12 : 'L_Elbow',
    13 : 'L_Wrist',
    14 : 'R_Shoulder',
    15 : 'R_Elbow',
    16 : 'R_Wrist',
}

In [13]:
def visualize_3d_pose(pose_list, dataset='h36m', xlim=(-512, 512), ylim=(-512, 512), zlim=(-512, 512), save=False, save_path='./', name='pose.png', elev=0., azim=90., dist=10):
    fig = plt.figure(0, figsize=(10, 10))
    ax = plt.axes(projection="3d")
    ax.clear()
    ax.set_proj_type('persp', focal_length=1)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=elev, azim=azim)
    ax.dist = dist
    for pose in pose_list:
        show3Dpose(pose, ax, dataset)
    if save:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        plt.savefig(os.path.join(save_path, name))
    else:
        plt.show()

# https://github.com/Vegetebird/MHFormer/blob/main/demo/vis.py
def show3Dpose(pose, ax, dataset='h36m'):
    if dataset == 'aihub':
        joint_pairs = [[15,12],  [12,17],  [12,16],  [17,19],  [19,21], [16,18], [18,20], [12,0], [0,1], [0,2], [2,1], [2,5], [5,8], [1,4], [4,7], [12,17]]
        joint_pairs_left = [[12,16], [16,18], [18,20], [0,1], [1,4], [4,7]]
        joint_pairs_right = [[12,17], [17,19],[19,21], [0,2], [2,5], [5,8]]
    else: # 'h36m'
        joint_pairs = [[0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6], [0, 7], [7, 8], [8, 9], [8, 11], [8, 14], [9, 10], [11, 12], [12, 13], [14, 15], [15, 16]]
        joint_pairs_left = [[8, 11], [11, 12], [12, 13], [0, 4], [4, 5], [5, 6]]
        joint_pairs_right = [[8, 14], [14, 15], [15, 16], [0, 1], [1, 2], [2, 3]]

    color_mid = "#00457E"
    color_left = "#02315E"
    color_right = "#2F70AF"

    j3d = pose
    
    # plt.tick_params(left = True, right = True , labelleft = False ,
    #                 labelbottom = False, bottom = False)
    # 좀 더 보기 좋게 하기 위해 y <-> z, - 붙임
    for i in range(len(joint_pairs)):
        limb = joint_pairs[i]
        if dataset == 'h36m':
            xs, zs, ys = [np.array([j3d[limb[0], j], j3d[limb[1], j]]) for j in range(3)]
            xs *= -1
            ys *= -1
            zs *= -1
            #xs, ys, zs = [np.array([j3d[limb[0], j], j3d[limb[1], j]]) for j in range(3)]
        elif dataset == 'aihub':
            xs, ys, zs = [np.array([j3d[limb[0], j], j3d[limb[1], j]]) for j in range(3)]
        if joint_pairs[i] in joint_pairs_left:
            ax.plot(xs, ys, zs, color=color_left, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
        elif joint_pairs[i] in joint_pairs_right:
            ax.plot(xs, ys, zs, color=color_right, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
        else:
            ax.plot(xs, ys, zs, color=color_mid, lw=3, marker='o', markerfacecolor='w', markersize=3, markeredgewidth=2) # axis transformation for visualization
        ax.axis('equal')
        #ax.axis('auto')


In [23]:
frame_num = 500
img_3d = copy.deepcopy(datareader['test']['joint3d_image'][frame_num])
img_2d = copy.deepcopy(datareader['test']['joint_2d'][frame_num])
cam_3d = copy.deepcopy(datareader['test']['cam_3d'][frame_num])
world_3d = copy.deepcopy(datareader['test']['world_3d'][frame_num])
frame = copy.deepcopy(datareader['test']['frame'][frame_num])
source = copy.deepcopy(datareader['test']['source'][frame_num])

In [24]:
source, frame

('res_30_F160A_1', 560)

In [26]:
img_3d = copy.deepcopy(datareader['test']['joint3d_image'][500])
#result_array[:, 2] = 0
r_hip = img_3d[1]
r_knee = img_3d[2]
r_ankle = img_3d[3]
l_hip = img_3d[4]
l_knee = img_3d[5]
l_ankle = img_3d[6]
#r_knee[2] += -100
visualize_3d_pose([world_3d], 
                  xlim=(0, 1920), 
                  ylim=(0, 1080),)
                  #elev=-90.,
                  #azim=-90+20.,
                  #dist=10.)

/tmp/ipykernel_7352/3109523603.py:13: MatplotlibDeprecationWarning: The dist attribute was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax.dist = dist


In [16]:
np.linalg.norm(l_hip - l_knee), np.linalg.norm(r_hip - r_knee)

(111.81644479325838, 113.25240136949434)

In [17]:
np.linalg.norm(l_knee - l_ankle), np.linalg.norm(r_knee - r_ankle)

(110.56688313937217, 109.81389674105407)